# Descuentos por volúmen | Utilización según tipo de negocio

En este trabajo nos proponemos caracterizar la forma en que los distintos negocios utilizan los descuentos por volumen. La pregunta de investigación es: ¿los distintos tipos de clientes utilizan de forma distinta los descuentos por volumen? 

Con el término 'la forma de utilización de los descuentos' nos referimos al tipo de producto comprado bajo un descuento de volumen, la cantidad de unidades compradas en promedio y el descuento particular utilizado.

Nuestras hipótesis pueden expresarse como:

- Los distintos tipos de cliente se diferencian en la forma de utilizar los descuentos por volumen
- El tipo de cliente influye en el tipo de producto comprado bajo una oferta por volumen
- El tipo de cliente influye en el promedio de unidades compradas bajo una oferta de volumen
- El tipo de cliente influye en el descuento particular utilizado de las ofertas de volumen

Nos enfocaremos en:

1. Caracterizar a los tipos de clientes de la empresa y su comportamiento de compra
2. Caracterizar a los descuentos por volumen (qué productos los aplican, cuántas unidades vendidas acumulan, promedio de unidades las líneas de las ordenes que los utilizan)
3. Caracterizar la relación entre los distintos tipos de clientes y los descuentos por volumen (qué tipos de productos compran, cuántas unidades compran en promedio y qué descuento es el que más utilizan)

1\. Caracterizar a los tipos de clientes de la empresa y su comportamiento de compra

In [18]:
WITH CTE AS (
    SELECT r.BusinessType, SalesOrderNumber,
    CONCAT(SalesOrderNumber, '-', SalesOrderLineNumber) AS LineId, 
    CASE WHEN PromotionKey IN(2,3,4,5,6) THEN 'VolProm' ELSE 'noVolProm' END volProm,
    OrderQuantity

    FROM AdventureWorksDW2019.dbo.FactResellerSales rs
    LEFT JOIN AdventureWorksDW2019.dbo.DimReseller r
    ON rs.ResellerKey = r.ResellerKey
)

SELECT DISTINCT BusinessType, volProm,
COUNT(LineId) OVER(PARTITION BY BusinessType, volProm) AS NumLines,
COUNT(LineId) OVER(PARTITION BY BusinessType) AS totalLines,
FORMAT(COUNT(LineId) OVER(PARTITION BY BusinessType, volProm) * 1.0 /  (COUNT(LineId) OVER(PARTITION BY BusinessType)),'P') AS percLines,
FORMAT(COUNT(LineId) OVER(PARTITION BY BusinessType) * 1.0 /  (COUNT(LineId) OVER()),'P') AS percTotalLines,
SUM(OrderQuantity) OVER(PARTITION BY BusinessType) AS NumUnits,
FORMAT(SUM(OrderQuantity) OVER(PARTITION BY BusinessType, volProm)* 1.0 /  (SUM(OrderQuantity) OVER()) ,'P') AS percUnits,
FORMAT(SUM(OrderQuantity) OVER(PARTITION BY BusinessType)* 1.0 /  (SUM(OrderQuantity) OVER()) ,'P') AS percTotalUnits,
AVG(OrderQuantity) OVER(PARTITION BY BusinessType, volProm) AS avgOrderQty,
AVG(OrderQuantity) OVER(PARTITION BY BusinessType) AS avgGralOrderQty


FROM CTE
ORDER BY BusinessType, volProm

(6 rows affected)

Total execution time: 00:00:58.384

BusinessType,volProm,NumLines,totalLines,percLines,percTotalLines,NumUnits,percUnits,percTotalUnits,avgOrderQty,avgGralOrderQty
Specialty Bike Shop,noVolProm,7909,7987,99.02%,13.12%,22076,9.76%,10.30%,2,2
Specialty Bike Shop,VolProm,78,7987,0.98%,13.12%,22076,0.54%,10.30%,14,2
Value Added Reseller,noVolProm,24698,25134,98.27%,41.30%,80309,34.06%,37.46%,2,3
Value Added Reseller,VolProm,436,25134,1.73%,41.30%,80309,3.40%,37.46%,16,3
Warehouse,noVolProm,26250,27734,94.65%,45.57%,111993,42.57%,52.24%,3,4
Warehouse,VolProm,1484,27734,5.35%,45.57%,111993,9.67%,52.24%,13,4


2\. Caracterizar a los descuentos por volumen (qué productos los aplican, cuántas unidades vendidas acumulan, promedio de unidades las líneas de las ordenes que los utilizan)

In [34]:
WITH CTE AS(
SELECT DISTINCT rs.ProductKey, pc.ProductCategoryKey, pc.EnglishProductCategoryName, ps.ProductSubcategoryKey, ps.EnglishProductSubcategoryName,
CASE WHEN PromotionKey IN(2,3,4,5,6) THEN 'VolProm' ELSE 'noVolProm' END volProm

FROM AdventureWorksDW2019.dbo.FactResellerSales rs
LEFT JOIN AdventureWorksDW2019.dbo.DimProduct dp
ON rs.ProductKey = dp.ProductKey
LEFT JOIN AdventureWorksDW2019.dbo.DimProductSubcategory ps
ON dp.ProductSubcategoryKey = ps.ProductSubcategoryKey
LEFT JOIN AdventureWorksDW2019.dbo.DimProductCategory pc 
ON ps.ProductCategoryKey = pc.ProductCategoryKey

)

SELECT DISTINCT EnglishProductCategoryName, EnglishProductSubcategoryName, ProductKey, volProm,
-- acá tendría que ser un count distinct pero eso no se permite en las window functions, rehacer de otra manera
COUNT(ProductKey) OVER(PARTITION BY ProductSubcategoryKey, volProm)  AS prodQty,
FORMAT(COUNT(ProductKey) OVER(PARTITION BY ProductSubcategoryKey, volProm) * 1.0 / COUNT(ProductKey) OVER(PARTITION BY ProductSubcategoryKey),'P')  AS percProd

FROM CTE
ORDER BY ProductKey

(473 rows affected)

Total execution time: 00:00:00.466

EnglishProductCategoryName,EnglishProductSubcategoryName,ProductKey,volProm,prodQty,percProd
Accessories,Helmets,212,noVolProm,9,52.94%
Accessories,Helmets,213,noVolProm,9,52.94%
Accessories,Helmets,213,VolProm,8,47.06%
Accessories,Helmets,214,noVolProm,9,52.94%
Accessories,Helmets,214,VolProm,8,47.06%
Accessories,Helmets,215,noVolProm,9,52.94%
Accessories,Helmets,215,VolProm,8,47.06%
Accessories,Helmets,216,noVolProm,9,52.94%
Accessories,Helmets,216,VolProm,8,47.06%
Accessories,Helmets,217,noVolProm,9,52.94%


In [35]:
SELECT DISTINCT rs.ProductKey, pc.ProductCategoryKey, pc.EnglishProductCategoryName, ps.ProductSubcategoryKey, ps.EnglishProductSubcategoryName,
CASE WHEN PromotionKey IN(2,3,4,5,6) THEN 'VolProm' ELSE 'noVolProm' END volProm

FROM AdventureWorksDW2019.dbo.FactResellerSales rs
LEFT JOIN AdventureWorksDW2019.dbo.DimProduct dp
ON rs.ProductKey = dp.ProductKey
LEFT JOIN AdventureWorksDW2019.dbo.DimProductSubcategory ps
ON dp.ProductSubcategoryKey = ps.ProductSubcategoryKey
LEFT JOIN AdventureWorksDW2019.dbo.DimProductCategory pc 
ON ps.ProductCategoryKey = pc.ProductCategoryKey

ORDER BY ProductKey

(473 rows affected)

Total execution time: 00:00:00.234

ProductKey,ProductCategoryKey,EnglishProductCategoryName,ProductSubcategoryKey,EnglishProductSubcategoryName,volProm
212,4,Accessories,31,Helmets,noVolProm
213,4,Accessories,31,Helmets,noVolProm
213,4,Accessories,31,Helmets,VolProm
214,4,Accessories,31,Helmets,noVolProm
214,4,Accessories,31,Helmets,VolProm
215,4,Accessories,31,Helmets,noVolProm
215,4,Accessories,31,Helmets,VolProm
216,4,Accessories,31,Helmets,noVolProm
216,4,Accessories,31,Helmets,VolProm
217,4,Accessories,31,Helmets,noVolProm


3\. Caracterizar la relación entre los distintos tipos de clientes y los descuentos por volumen (qué tipos de productos compran, cuántas unidades compran en promedio y qué descuento es el que más utilizan)

In [21]:
WITH CTE AS (
    SELECT rs.ProductKey, pc.ProductCategoryKey, pc.EnglishProductCategoryName, ps.ProductSubcategoryKey, ps.EnglishProductSubcategoryName,
    r.BusinessType, SalesOrderNumber,
    CONCAT(SalesOrderNumber, '-', SalesOrderLineNumber) AS LineId, 
    CASE WHEN PromotionKey IN(2,3,4,5,6) THEN 'VolProm' ELSE 'noVolProm' END volProm,
    OrderQuantity

    FROM AdventureWorksDW2019.dbo.FactResellerSales rs
    LEFT JOIN AdventureWorksDW2019.dbo.DimReseller r
    ON rs.ResellerKey = r.ResellerKey
    LEFT JOIN AdventureWorksDW2019.dbo.DimProduct dp
    ON rs.ProductKey = dp.ProductKey
    LEFT JOIN AdventureWorksDW2019.dbo.DimProductSubcategory ps
    ON dp.ProductSubcategoryKey = ps.ProductSubcategoryKey
    LEFT JOIN AdventureWorksDW2019.dbo.DimProductCategory pc 
    ON ps.ProductCategoryKey = pc.ProductCategoryKey
)

SELECT DISTINCT BusinessType, volProm, EnglishProductCategoryName,
COUNT(LineId) OVER(PARTITION BY BusinessType, volProm, ProductCategoryKey) AS NumLines,
COUNT(LineId) OVER(PARTITION BY BusinessType, ProductCategoryKey) AS totalLines,
FORMAT(COUNT(LineId) OVER(PARTITION BY BusinessType, volProm, ProductCategoryKey) * 1.0 /  (COUNT(LineId) OVER(PARTITION BY BusinessType, ProductCategoryKey)), 'P') AS percLines,

FORMAT(COUNT(LineId) OVER(PARTITION BY BusinessType, ProductCategoryKey) * 1.0 /  (COUNT(LineId) OVER(PARTITION BY BusinessType)),'P') AS percTotalLines,

SUM(OrderQuantity) OVER(PARTITION BY BusinessType, volProm, ProductCategoryKey) AS NumUnits,
FORMAT(SUM(OrderQuantity) OVER(PARTITION BY BusinessType, volProm, ProductCategoryKey)* 1.0 /  SUM(OrderQuantity) OVER(PARTITION BY BusinessType, ProductCategoryKey),'P') AS percUnits,
AVG(OrderQuantity) OVER(PARTITION BY BusinessType, volProm, ProductCategoryKey) AS avgOrderQty



FROM CTE
ORDER BY BusinessType, EnglishProductCategoryName, volProm

(21 rows affected)

Total execution time: 00:00:44.723

BusinessType,volProm,EnglishProductCategoryName,NumLines,totalLines,percLines,percTotalLines,NumUnits,percUnits,avgOrderQty
Specialty Bike Shop,noVolProm,Accessories,749,749,100.00%,9.38%,2392,100.00%,3
Specialty Bike Shop,noVolProm,Bikes,3427,3427,100.00%,42.91%,7094,100.00%,2
Specialty Bike Shop,noVolProm,Clothing,2580,2658,97.07%,33.28%,8857,88.40%,3
Specialty Bike Shop,VolProm,Clothing,78,2658,2.93%,33.28%,1162,11.60%,14
Specialty Bike Shop,noVolProm,Components,1153,1153,100.00%,14.44%,2571,100.00%,2
Value Added Reseller,noVolProm,Accessories,1946,1955,99.54%,7.78%,7662,98.53%,3
Value Added Reseller,VolProm,Accessories,9,1955,0.46%,7.78%,114,1.47%,12
Value Added Reseller,noVolProm,Bikes,12319,12355,99.71%,49.16%,34077,98.75%,2
Value Added Reseller,VolProm,Bikes,36,12355,0.29%,49.16%,432,1.25%,12
Value Added Reseller,noVolProm,Clothing,3672,4057,90.51%,16.14%,15125,69.42%,4


In [29]:
WITH CTE AS (
    SELECT rs.ProductKey, pc.ProductCategoryKey, pc.EnglishProductCategoryName, ps.ProductSubcategoryKey, ps.EnglishProductSubcategoryName,
    r.BusinessType, SalesOrderNumber,
    CONCAT(SalesOrderNumber, '-', SalesOrderLineNumber) AS LineId, 
    CASE WHEN PromotionKey IN(2,3,4,5,6) THEN 'VolProm' ELSE 'noVolProm' END volProm,
    OrderQuantity

    FROM AdventureWorksDW2019.dbo.FactResellerSales rs
    LEFT JOIN AdventureWorksDW2019.dbo.DimReseller r
    ON rs.ResellerKey = r.ResellerKey
    LEFT JOIN AdventureWorksDW2019.dbo.DimProduct dp
    ON rs.ProductKey = dp.ProductKey
    LEFT JOIN AdventureWorksDW2019.dbo.DimProductSubcategory ps
    ON dp.ProductSubcategoryKey = ps.ProductSubcategoryKey
    LEFT JOIN AdventureWorksDW2019.dbo.DimProductCategory pc 
    ON ps.ProductCategoryKey = pc.ProductCategoryKey
)

SELECT DISTINCT BusinessType, volProm, EnglishProductCategoryName, EnglishProductSubcategoryName,
COUNT(LineId) OVER(PARTITION BY BusinessType, volProm, ProductSubcategoryKey) AS NumLines,
COUNT(LineId) OVER(PARTITION BY BusinessType, ProductSubcategoryKey) AS totalLines,
FORMAT(COUNT(LineId) OVER(PARTITION BY BusinessType, volProm, ProductSubcategoryKey) * 1.0 /  (COUNT(LineId) OVER(PARTITION BY BusinessType, ProductSubcategoryKey)), 'P') AS percLines,

FORMAT(COUNT(LineId) OVER(PARTITION BY BusinessType, ProductSubcategoryKey) * 1.0 /  (COUNT(LineId) OVER(PARTITION BY BusinessType)),'P') AS percTotalLines,

SUM(OrderQuantity) OVER(PARTITION BY BusinessType, volProm, ProductSubcategoryKey) AS NumUnits,
FORMAT(SUM(OrderQuantity) OVER(PARTITION BY BusinessType, volProm, ProductSubcategoryKey)* 1.0 /  SUM(OrderQuantity) OVER(PARTITION BY BusinessType, ProductSubcategoryKey),'P') AS percUnits,
AVG(OrderQuantity) OVER(PARTITION BY BusinessType, volProm, ProductSubcategoryKey) AS avgOrderQty



FROM CTE
--ORDER BY BusinessType, EnglishProductCategoryName, ProductSubcategoryKey, volProm

(149 rows affected)

Total execution time: 00:00:44.599

BusinessType,volProm,EnglishProductCategoryName,EnglishProductSubcategoryName,NumLines,totalLines,percLines,percTotalLines,NumUnits,percUnits,avgOrderQty
Specialty Bike Shop,noVolProm,Accessories,Bike Racks,144,144,100.00%,1.80%,510,100.00%,3
Specialty Bike Shop,noVolProm,Accessories,Bottles and Cages,121,121,100.00%,1.51%,388,100.00%,3
Specialty Bike Shop,noVolProm,Accessories,Cleaners,99,99,100.00%,1.24%,298,100.00%,3
Specialty Bike Shop,noVolProm,Accessories,Helmets,336,336,100.00%,4.21%,1019,100.00%,3
Specialty Bike Shop,noVolProm,Accessories,Hydration Packs,47,47,100.00%,0.59%,175,100.00%,3
Specialty Bike Shop,noVolProm,Accessories,Pumps,2,2,100.00%,0.03%,2,100.00%,1
Specialty Bike Shop,noVolProm,Bikes,Mountain Bikes,852,852,100.00%,10.67%,1691,100.00%,1
Specialty Bike Shop,noVolProm,Bikes,Road Bikes,1770,1770,100.00%,22.16%,4023,100.00%,2
Specialty Bike Shop,noVolProm,Bikes,Touring Bikes,805,805,100.00%,10.08%,1380,100.00%,1
Specialty Bike Shop,noVolProm,Clothing,Bib-Shorts,54,55,98.18%,0.69%,156,93.41%,2
